In [1]:
import pandas as pd

/tmp/ipykernel_394633/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_excel('Axis Bank Statement.xlsx')
df = df.drop(df.index[[0, -1, -2]])
df = df.drop(columns=['Chq No', 'Init. Br'])

df

,Tran Date,Particulars,Debit,Credit,Balance
1,2023-06-30,UPI/P2M/318123361742/EURONETGP/ICICI\nBan/UPI,61.0,NaN,20018.76
2,2023-06-30,UPI/P2A/318165871904/M THANGAR/Union\nBan/Payment,100.0,NaN,19918.76
3,2023-06-30,UPI/P2M/318133847912/Sri Krish/Paytm Pay/UPI,45.0,NaN,19873.76
4,2023-06-30,UPI/P2A/318137076154/Ms E V R /City\nUnio/UPI,350.0,NaN,19523.76
5,2023-06-30,UPI/P2A/318199079945/VENMATHI /State\nBan/UPI,NaN,11380.0,30903.76
...,...,...,...,...,...
1503,2024-01-05,UPI/P2M/400529263423/JUST CAKES\n/Paymen/Paytm...,70.0,NaN,12325.00
1504,2024-01-05,UPI/P2M/400579109544/BharatPe Merchant\n/Pay t...,100.0,NaN,12225.00
1505,2024-01-05,UPI/P2A/400535601226/RAVICHAND/Karur\nVys/UPI/,NaN,1000.0,13225.00
1506,2024-01-05,UPI/P2M/400591314408/Simpl /UPI...,3950.0,NaN,9275.00


In [3]:
# Convert 'Tran Date' to datetime format (if not already done)
df['Tran Date'] = pd.to_datetime(df['Tran Date'])

# Create new columns for Year, Month, Day, and Weekday
df['Year'] = df['Tran Date'].dt.year
df['Month'] = df['Tran Date'].dt.month
df['Day'] = df['Tran Date'].dt.day
df['Weekday'] = df['Tran Date'].dt.day_name()

# Display the updated dataframe to verify the changes
df[['Tran Date', 'Year', 'Month', 'Day', 'Weekday']].head()


,Tran Date,Year,Month,Day,Weekday
1,2023-06-30,2023,6,30,Friday
2,2023-06-30,2023,6,30,Friday
3,2023-06-30,2023,6,30,Friday
4,2023-06-30,2023,6,30,Friday
5,2023-06-30,2023,6,30,Friday


In [4]:
# Create 'Transaction' and 'Type of Transaction' columns
df['Transaction'] = df['Debit'].fillna(0) - df['Credit'].fillna(0)
df['Type of Transaction'] = df['Transaction'].apply(lambda x: 'Debit' if x > 0 else 'Credit')

# Make 'Transaction' column always positive for percentile calculation
df['Transaction'] = df['Transaction'].abs()

# Calculate percentiles for the 'Transaction' column
percentiles = df['Transaction'].quantile([0.15, 0.50, 0.80]).values

# Function to categorize the size of the transaction
def categorize_transaction_size(amount):
    if amount <= percentiles[0]:
        return 'Small'
    elif amount <= percentiles[1]:
        return 'Medium'
    elif amount <= percentiles[2]:
        return 'Large'
    else:
        return 'Huge'

# Apply the function to categorize 'Transaction' sizes
df['Transaction Size'] = df['Transaction'].apply(categorize_transaction_size)

# Display the updated dataframe to verify the changes
df[['Debit', 'Credit', 'Transaction', 'Type of Transaction', 'Transaction Size']].head()


,Debit,Credit,Transaction,Type of Transaction,Transaction Size
1,61.0,NaN,61.0,Debit,Medium
2,100.0,NaN,100.0,Debit,Medium
3,45.0,NaN,45.0,Debit,Medium
4,350.0,NaN,350.0,Debit,Large
5,NaN,11380.0,11380.0,Credit,Huge


In [5]:
df

,Tran Date,Particulars,Debit,Credit,Balance,Year,Month,Day,Weekday,Transaction,Type of Transaction,Transaction Size
1,2023-06-30,UPI/P2M/318123361742/EURONETGP/ICICI\nBan/UPI,61.0,NaN,20018.76,2023,6,30,Friday,61.0,Debit,Medium
2,2023-06-30,UPI/P2A/318165871904/M THANGAR/Union\nBan/Payment,100.0,NaN,19918.76,2023,6,30,Friday,100.0,Debit,Medium
3,2023-06-30,UPI/P2M/318133847912/Sri Krish/Paytm Pay/UPI,45.0,NaN,19873.76,2023,6,30,Friday,45.0,Debit,Medium
4,2023-06-30,UPI/P2A/318137076154/Ms E V R /City\nUnio/UPI,350.0,NaN,19523.76,2023,6,30,Friday,350.0,Debit,Large
5,2023-06-30,UPI/P2A/318199079945/VENMATHI /State\nBan/UPI,NaN,11380.0,30903.76,2023,6,30,Friday,11380.0,Credit,Huge
...,...,...,...,...,...,...,...,...,...,...,...,...
1503,2024-01-05,UPI/P2M/400529263423/JUST CAKES\n/Paymen/Paytm...,70.0,NaN,12325.00,2024,1,5,Friday,70.0,Debit,Medium
1504,2024-01-05,UPI/P2M/400579109544/BharatPe Merchant\n/Pay t...,100.0,NaN,12225.00,2024,1,5,Friday,100.0,Debit,Medium
1505,2024-01-05,UPI/P2A/400535601226/RAVICHAND/Karur\nVys/UPI/,NaN,1000.0,13225.00,2024,1,5,Friday,1000.0,Credit,Huge
1506,2024-01-05,UPI/P2M/400591314408/Simpl /UPI...,3950.0,NaN,9275.00,2024,1,5,Friday,3950.0,Debit,Huge


In [6]:
# # Split the 'Particulars' column by the "/" symbol
# split_particulars = df['Particulars'].str.split('/')

# # Extract the fourth element (index 3) from the split results
# df['To/From'] = split_particulars.apply(lambda x: x[3] if len(x) > 3 else None)

# # Display the updated dataframe to verify the changes
# df[['Particulars', 'To/From']].head()

# # Split the 'Particulars' column by the "/" symbol
# split_particulars = df['Particulars'].str.split('/')

# Assuming 'df' is your DataFrame and 'Particulars' column exists

# Split the 'Particulars' column by the "/" symbol
split_particulars = df['Particulars'].str.split('/')

# Extract components based on their positions
df['Variable'] = split_particulars.apply(lambda x: x[0] if len(x) > 0 else None)
df['Person-to-'] = split_particulars.apply(lambda x: x[1] if len(x) > 1 else None)
df['Person-to-'] = df['Person-to-'].replace({'P2A': 'Account', 'P2M': 'Merchant'})
df['Transaction ID'] = split_particulars.apply(lambda x: x[2] if len(x) > 2 else None)
df['Merchant or Recipient Identifier'] = split_particulars.apply(lambda x: x[3] if len(x) > 3 else None)
df['Bank Identifier'] = split_particulars.apply(lambda x: x[4] if len(x) > 4 else None)
df['Transaction Method'] = split_particulars.apply(lambda x: x[5] if len(x) > 5 else None)

# Display the updated dataframe to verify the changes
df[['Particulars', 'Variable', 'Person-to-', 'Transaction ID', 'Merchant or Recipient Identifier', 'Bank Identifier', 'Transaction Method']].head()


,Particulars,Variable,Person-to-,Transaction ID,Merchant or Recipient Identifier,Bank Identifier,Transaction Method
1,UPI/P2M/318123361742/EURONETGP/ICICI\nBan/UPI,UPI,Merchant,318123361742,EURONETGP,ICICI\nBan,UPI
2,UPI/P2A/318165871904/M THANGAR/Union\nBan/Payment,UPI,Account,318165871904,M THANGAR,Union\nBan,Payment
3,UPI/P2M/318133847912/Sri Krish/Paytm Pay/UPI,UPI,Merchant,318133847912,Sri Krish,Paytm Pay,UPI
4,UPI/P2A/318137076154/Ms E V R /City\nUnio/UPI,UPI,Account,318137076154,Ms E V R,City\nUnio,UPI
5,UPI/P2A/318199079945/VENMATHI /State\nBan/UPI,UPI,Account,318199079945,VENMATHI,State\nBan,UPI


In [7]:
df

,Tran Date,Particulars,Debit,Credit,Balance,Year,Month,Day,Weekday,Transaction,Type of Transaction,Transaction Size,Variable,Person-to-,Transaction ID,Merchant or Recipient Identifier,Bank Identifier,Transaction Method
1,2023-06-30,UPI/P2M/318123361742/EURONETGP/ICICI\nBan/UPI,61.0,NaN,20018.76,2023,6,30,Friday,61.0,Debit,Medium,UPI,Merchant,318123361742,EURONETGP,ICICI\nBan,UPI
2,2023-06-30,UPI/P2A/318165871904/M THANGAR/Union\nBan/Payment,100.0,NaN,19918.76,2023,6,30,Friday,100.0,Debit,Medium,UPI,Account,318165871904,M THANGAR,Union\nBan,Payment
3,2023-06-30,UPI/P2M/318133847912/Sri Krish/Paytm Pay/UPI,45.0,NaN,19873.76,2023,6,30,Friday,45.0,Debit,Medium,UPI,Merchant,318133847912,Sri Krish,Paytm Pay,UPI
4,2023-06-30,UPI/P2A/318137076154/Ms E V R /City\nUnio/UPI,350.0,NaN,19523.76,2023,6,30,Friday,350.0,Debit,Large,UPI,Account,318137076154,Ms E V R,City\nUnio,UPI
5,2023-06-30,UPI/P2A/318199079945/VENMATHI /State\nBan/UPI,NaN,11380.0,30903.76,2023,6,30,Friday,11380.0,Credit,Huge,UPI,Account,318199079945,VENMATHI,State\nBan,UPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,2024-01-05,UPI/P2M/400529263423/JUST CAKES\n/Paymen/Paytm...,70.0,NaN,12325.00,2024,1,5,Friday,70.0,Debit,Medium,UPI,Merchant,400529263423,JUST CAKES\n,Paymen,Paytm Payments Bank
1504,2024-01-05,UPI/P2M/400579109544/BharatPe Merchant\n/Pay t...,100.0,NaN,12225.00,2024,1,5,Friday,100.0,Debit,Medium,UPI,Merchant,400579109544,BharatPe Merchant\n,Pay to,YesBank_Yespay
1505,2024-01-05,UPI/P2A/400535601226/RAVICHAND/Karur\nVys/UPI/,NaN,1000.0,13225.00,2024,1,5,Friday,1000.0,Credit,Huge,UPI,Account,400535601226,RAVICHAND,Karur\nVys,UPI
1506,2024-01-05,UPI/P2M/400591314408/Simpl /UPI...,3950.0,NaN,9275.00,2024,1,5,Friday,3950.0,Debit,Huge,UPI,Merchant,400591314408,Simpl,UPI\nIn,Axis Bank Ltd.


In [11]:
df_counts = df['Merchant or Recipient Identifier'].value_counts()
df_repeated = df[df['Merchant or Recipient Identifier'].isin(df_counts[df_counts > 5].index)]
unique_values = df_repeated['Merchant or Recipient Identifier'].unique()
print(unique_values)


['EURONETGP' 'Sri Krish' 'Ms E V R ' 'VENMATHI ' 'Sai South' 'BharatPe\n'
 'S Joshik ' 'RUPESH RA' 'BALAMURUG' 'PayU Paym' 'CHEESE ST' 'UMARANI  '
 'Simpl' 'NICKILVIS' 'RAHUL R C' ' UDUPI GA' 'Bangalore' 'Spotify I'
 'RAVICHAND' 'Apple Ser' 'WESTSIDE ' 'C SRIDHAR\n'
 'Ms E V R\nVishwadharan' 'Sri Krishna Cafe\n' 'Bangalore Metro Rail\n'
 'BharatPe Merchant\n' 'Jio Prepaid Recharges' 'RAHUL R CHANDAN\n'
 'Master AR' 'Simpl                ' 'NICKILVISHWAA  MS\n' 'EURONETGPAY\n'
 'ZOXO FRESH MART\n' 'CHEESE STREET\n' 'billdesktez\n'
 'SmartPoint Mathikere\n' 'JUST CAKES\n' 'RAJARAM KAMAT\n'
 'MAHAMMED ANSAR\n' 'By 2 cafe\n' 'PREMA                ' 'SWIGGY\n'
 'PAYU PAYMENTS PVT. LT' 'E R K  JA' 'E R K  JAIKAUSHIK\n' 'BMTC\n'
 'Bengaluru Metropolita']


In [12]:
# Load the holidays in Tamil Nadu Excel file to extract the list of holidays
tn_holidays_path = 'Holidays in Tamil Nadu 2023 - ExcelNotes.xlsx'
tn_holidays_df = pd.read_excel(tn_holidays_path)

# Display the first few rows of the holidays dataframe to understand its structure
tn_holidays_df.head()

,Holiday,Date,Weekday,Note
0,New Year's Day,"Jan 01, 2023",Sunday,Regional Holiday
1,Pongal,"Jan 15, 2023",Sunday,Regional Holiday
2,Thiruvalluvar Day,"Jan 16, 2023",Monday,Regional Holiday
3,Uzhavar Thirunal,"Jan 17, 2023",Tuesday,Regional Holiday
4,Republic Day,"Jan 26, 2023",Thursday,Public Holiday


In [13]:
df

,Tran Date,Particulars,Debit,Credit,Balance,Year,Month,Day,Weekday,Transaction,Type of Transaction,Transaction Size,Variable,Person-to-,Transaction ID,Merchant or Recipient Identifier,Bank Identifier,Transaction Method
1,2023-06-30,UPI/P2M/318123361742/EURONETGP/ICICI\nBan/UPI,61.0,NaN,20018.76,2023,6,30,Friday,61.0,Debit,Medium,UPI,Merchant,318123361742,EURONETGP,ICICI\nBan,UPI
2,2023-06-30,UPI/P2A/318165871904/M THANGAR/Union\nBan/Payment,100.0,NaN,19918.76,2023,6,30,Friday,100.0,Debit,Medium,UPI,Account,318165871904,M THANGAR,Union\nBan,Payment
3,2023-06-30,UPI/P2M/318133847912/Sri Krish/Paytm Pay/UPI,45.0,NaN,19873.76,2023,6,30,Friday,45.0,Debit,Medium,UPI,Merchant,318133847912,Sri Krish,Paytm Pay,UPI
4,2023-06-30,UPI/P2A/318137076154/Ms E V R /City\nUnio/UPI,350.0,NaN,19523.76,2023,6,30,Friday,350.0,Debit,Large,UPI,Account,318137076154,Ms E V R,City\nUnio,UPI
5,2023-06-30,UPI/P2A/318199079945/VENMATHI /State\nBan/UPI,NaN,11380.0,30903.76,2023,6,30,Friday,11380.0,Credit,Huge,UPI,Account,318199079945,VENMATHI,State\nBan,UPI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,2024-01-05,UPI/P2M/400529263423/JUST CAKES\n/Paymen/Paytm...,70.0,NaN,12325.00,2024,1,5,Friday,70.0,Debit,Medium,UPI,Merchant,400529263423,JUST CAKES\n,Paymen,Paytm Payments Bank
1504,2024-01-05,UPI/P2M/400579109544/BharatPe Merchant\n/Pay t...,100.0,NaN,12225.00,2024,1,5,Friday,100.0,Debit,Medium,UPI,Merchant,400579109544,BharatPe Merchant\n,Pay to,YesBank_Yespay
1505,2024-01-05,UPI/P2A/400535601226/RAVICHAND/Karur\nVys/UPI/,NaN,1000.0,13225.00,2024,1,5,Friday,1000.0,Credit,Huge,UPI,Account,400535601226,RAVICHAND,Karur\nVys,UPI
1506,2024-01-05,UPI/P2M/400591314408/Simpl /UPI...,3950.0,NaN,9275.00,2024,1,5,Friday,3950.0,Debit,Huge,UPI,Merchant,400591314408,Simpl,UPI\nIn,Axis Bank Ltd.
